In [30]:
import numpy as np
import pandas as pd
from numpy import sqrt,arctan,arccos,arcsin,sin,cos,tan,mod,pi,mean

#Conversion of Degree <--> Radians
def deg2rad(x):
    return x*pi/180
def rad2deg(x):
    return x*180/pi

# Change from spherical to cartesian coordinate
def get_cart(lon,lat,r=1):
    phi = pi/2.-deg2rad(float(lat))
    theta = deg2rad(float(lon))
    x = r*sin(phi)*cos(theta)
    y = r*sin(phi)*sin(theta)
    z = r*cos(phi)
    return (x,y,z)

# Change from cartisan to spherical coordinate
def get_sph(x,y,z):
    r = sqrt(x*x+y*y+z*z)
    phi = arccos(z/r)
    theta = arcsin(y/r/sin(phi))
    lon = rad2deg(theta)
    lat = rad2deg(pi/2.-phi)
    if lat > 90.:
        lat = lat-180.
    return (r,lon,lat)

# Calculate centeriod of data points in spherical coordinate
# arr is pandas data frame
def get_centeroid(arr):
    npts = 0
    xc,yc,zc = 0.,0.,0.
    for i in range(0,len(arr)):
        cart = get_cart(arr["lon"][i],arr['lat'][i],r=arr['r'][i])
        xc += cart[0]
        yc += cart[1]
        zc += cart[2]
        npts+=1
    out = get_sph(xc/npts,yc/npts,zc/npts)
    C = pd.DataFrame.from_dict([{"lon":out[1],"lat":out[2],"r":out[0]}])
    return C

# Calculate distance of points from centeroid
def get_ang_sep(P1,P2):
    (x1,y1,z1) = get_cart(P1["lon"],P1["lat"],P1["r"])
    (x2,y2,z2) = get_cart(P2["lon"],P2["lat"],P2["r"])
    angle = (x1*x2+y1*y2+z1*z2)/sqrt(x1**2+y1**2+z1**2)/sqrt(x2**2+y2**2+z2**2)
    angle = arccos(angle)
    return rad2deg(angle)

# Make pandas data frame of coordinates
def stx2pandas(coords):
    arr = pd.DataFrame.from_dict(coords)
    return arr

# Wrapper program
def calculate_centeroid(coords,min_fov=5):

    for i in range(len(coords["lat"])):
        lat = coords["lat"][i]
        if lat < -90. or lat >+90.:
            raise Exception("Range Error. -90 <= Latitude <= +90")

    try:
        coords["r"]
    except:
        coords["r"]=[1. for i in range(len(coords["lat"]))]
    arr = stx2pandas(coords)
    cent = get_centeroid(arr)
    df = arr.copy().reset_index()
    df["dist"]=[0. for i in range(len(coords["lat"]))]
    
    for i in range(len(df)):
        P = df.iloc[[i]].reset_index()
        dist = get_ang_sep(P,cent)
        df.loc[[i],"dist"] = [dist]
    
    max_dist = max(df.dist)
    dist_round = min_fov*round(max_dist/min_fov)
    if dist_round < max_dist:
        dist_round+=min_fov
        
    out = {"lon": cent["lon"],
           "lat":cent["lat"],
           "fov":[dist_round]}
    out = pd.DataFrame.from_dict(out)
    return out

In [32]:
gal = {"lon":[248,235.2,240,257.1,237.4,266.2,261.3],
       "lat":[-49.2,-49.1,-48.6,-48.4,-48.2,-44.4,-40.2],
       }
out = calculate_centeroid(gal)
out

,lon,lat,fov
0,-69.76462,-47.455927,40
